In [3]:
import dotenv as dt
import pandas as pd
from dotenv import load_dotenv
from openbb_terminal.sdk import openbb as ob

load_dotenv("./api.env")

True

In [4]:
eo_login = ob.login(
    email=f"{dt.get_key(dotenv_path='api.env', key_to_get='EMAIL')}",
    password=f"{dt.get_key(dotenv_path='api.env', key_to_get='PASSWORD')}",
)
eo_login

Creating session from email and password.

Login successful.

In [9]:
ob.futures.search("CL")

,Ticker,Description,Exchange,Category


In [12]:
__cl = ob.futures.curve("CL")[0:3]
print(__cl)

          Last Price
Sep-2023   80.239998
Oct-2023   79.730003
Nov-2023   79.239998


No pricing data for selected date.

None


In [13]:
_cl = ob.futures.curve_chart("CL", date="2023-07-28")
print(_cl)

No pricing data for selected date.

None


In [9]:
chains = ["OXY", "AAPL", "TTD", "UPST", "MSFT", "GOOG"]



for chain in chains:
    # df = ob.stocks.options.load_options_chains(symbol=chain, source="CBOE", date="2023-07-28")
    df = ob.stocks.options.load_options_chains(symbol=chain, source="CBOE", pydantic=True)
    chain_s = pd.DataFrame(df.chains)
    print(f"chains:{chains_s}: data{chain_s[chain_s["expiration"] == df.expirations[-1]]}")


# df = ob.stocks.load(
#     [
#         "MSFT",
#         "GOOG",
#         "AAPL",
#         "UPST",
#         "TTD",
#         "OXY",
#     ],
#     start_date="2020-01-01",
# )

SyntaxError: f-string: unmatched '[' (2944926151.py, line 9)

In [3]:
# ob.ta.standard_deviation(df)

date
2020-02-14    0.118138
2020-02-18    0.115729
2020-02-19    0.115933
2020-02-20    0.116413
2020-02-21    0.120656
                ...   
2023-07-19    0.089640
2023-07-20    0.093282
2023-07-21    0.092067
2023-07-24    0.091590
2023-07-25    0.091776
Name: Close, Length: 866, dtype: float64

In [4]:
# df.head(1)

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
date,,,,,,,,
2020-01-02,306.117305,307.394613,305.161684,307.375671,307.375671,59151200,0.0,0.0


In [5]:
# rvol = ob.ta.rvol_std(df, window=1).rename("Trading Period 252")
# ob.qa.line(data=rvol, title="Rolling 30d Realized Vol - STD Model", log_y=False)

Error: Window must be at least 2, defaulting to 30.


In [6]:
# Compare the difference between annualizing over 252 trading days versus 365 calendar days.
# rvol = pd.concat(
#     [
#         rvol,
#         ob.ta.rvol_std(df, window=1, is_crypto=True, trading_periods=365).rename(
#             "Trading Period 365"
#         ),
#     ],
#     axis=1,
# )

# ob.forecast.plot(data=rvol, columns=rvol.columns)

Error: Window must be at least 2, defaulting to 30.
is_crypto is overridden by trading_periods.


Comparing the models against each other:
The other models work in exactly the same way. The default state for each model is a 30-day rolling window, annualized over 252 trading days.


In [7]:
# rvol = pd.DataFrame()
# rvol["STD Model"] = ob.ta.standard_deviation(df)
# rvol["Parkinson"] = ob.ta.rvol_parkinson(df)
# rvol["Hodges-Tompkins"] = ob.ta.rvol_hodges_tompkins(df)
# rvol["Garman-Klass"] = ob.ta.rvol_garman_klass(df)
# rvol["Rogers-Satchell"] = ob.ta.rvol_rogers_satchell(df)
# rvol["Yang-Zhang"] = ob.ta.rvol_yang_zhang(df)

# ob.forecast.plot(data=rvol["2021-06-01":], columns=rvol.columns)

Generate realized volatility quantiles. Two functions, one for returning the dataframe only, other is for returning a chart.


In [25]:
# ob.ta.cones(df)

,Realized,Min,Lower 25%,Median,Upper 75%,Max
3,0.090944,0.008066,0.080743,0.143929,0.225367,1.747958
10,0.083900,0.049646,0.117136,0.170094,0.227689,1.130633
30,0.118139,0.075345,0.131229,0.177105,0.245596,0.826918
60,0.562053,0.089297,0.142257,0.179263,0.238623,0.617907
90,0.504496,0.096195,0.143275,0.182986,0.245525,0.525428
120,0.456182,0.101805,0.137188,0.183716,0.241471,0.460177
150,0.413430,0.105788,0.146181,0.189783,0.241676,0.419825
180,0.384724,0.107240,0.145150,0.193647,0.234356,0.392665
210,0.364282,0.119091,0.146874,0.194623,0.234010,0.367248
240,0.344880,0.120015,0.150275,0.195958,0.235934,0.346609


In [26]:
# ob.ta.cones_chart(df, symbol="SPY")

In [27]:
# ob.ta.cones_chart(df, model="Yang-Zhang", symbol="SPY")

In [28]:
# data input can also be the 'load' function from the openbb sdk. Use the symbol arg to complete the title of the chart.
# ob.ta.cones_chart(
#     data=ob.stocks.load("SPY"),
#     symbol="SPY",
#     model="Garman-Klass",
#     upper_q=0.90,
#     lower_q=0.10,
# )

Loading Daily data for SPY with starting period 2020-06-08.

Overlaying against IV
SPY is a good symbol for this as it has an index (VIX) that tracks the 30d IV of the around the ITM options. Using it as a proxy you can compare historical vs realized vol. Similarly to how the df was constructed to compare all the models together, I'll make a function with symbol input.


In [29]:
# from typing import Optional


# def rvol(data, window, trading_periods, is_crypto) -> pd.DataFrame:
#     results = pd.DataFrame()
#     results["Standard Deviation"] = ob.ta.rvol_std(
#         data, window, trading_periods, is_crypto
#     )
#     results["Parkinson"] = ob.ta.rvol_parkinson(
#         data, window, trading_periods, is_crypto
#     )
#     results["Hodges-Tompkins"] = ob.ta.rvol_hodges_tompkins(
#         data, window, trading_periods, is_crypto
#     )
#     results["Garman-Klass"] = ob.ta.rvol_garman_klass(
#         data, window, trading_periods, is_crypto
#     )
#     results["Rogers-Satchell"] = ob.ta.rvol_rogers_satchell(
#         data, window, trading_periods, is_crypto
#     )
#     results["Yang-Zhang"] = ob.ta.rvol_yang_zhang(
#         data, window, trading_periods, is_crypto
#     )
# return results


# def realized_vol(
#     symbol,
#     window: Optional[int] = 30,
#     trading_periods: Optional[int] = None,
#     is_crypto: Optional[bool] = False,
# ) -> pd.DataFrame:
#     rvol_df = rvol(ob.stocks.load(f"{symbol}"), window, trading_periods, is_crypto)

# return rvol_df

In [30]:
# data = realized_vol("SPY")
# data

Loading Daily data for SPY with starting period 2020-06-08.

,Standard Deviation,Parkinson,Hodges-Tompkins,Garman-Klass,Rogers-Satchell,Yang-Zhang
date,,,,,,
2020-07-21,0.247971,0.190572,0.258447,0.183398,0.182089,0.261515
2020-07-22,0.247299,0.189934,0.257746,0.182078,0.180378,0.259068
2020-07-23,0.249421,0.191085,0.259958,0.183212,0.181421,0.259940
2020-07-24,0.174964,0.178795,0.182356,0.177042,0.178622,0.240348
2020-07-27,0.173265,0.169015,0.180585,0.164655,0.163626,0.218372
...,...,...,...,...,...,...
2023-06-07,0.134154,0.098365,0.139822,0.097782,0.098507,0.123979
2023-06-08,0.133542,0.098026,0.139184,0.097299,0.098001,0.123627
2023-06-09,0.122511,0.095547,0.127686,0.096787,0.098457,0.120644


In [31]:
# data["VIX"] = ob.stocks.load("^VIX")["Adj Close"].rename("VIX") / 100

Loading Daily data for ^VIX with starting period 2020-06-08.

In [32]:
# fig = ob.forecast.plot(data, columns=data.columns, external_axes=True)

In [33]:
# fig.update_layout(
#     {
#         "title": "SPY - 30d Realized vs. Implied Volatility",
#         "title_y": 0.95,
#         "title_x": 0.5,
#     },
#     legend=dict(
#         yanchor="top",
#         y=1,
#         xanchor="right",
#         orientation="h",
#     ),
# )

In [34]:
# %%capture
# fig = ob.forecast.nbeats_chart(
# 	data = data,
# 	target_column = "VIX",
# 	past_covariates = "Standard Deviation, Parkinson, Hodges-Tompkins, Garman-Klass, Rogers-Satchell, Yang-Zhang",
# 	forecast_only = True,
# 	external_axes = True
# )

In [35]:
# help(fig)

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      True if self else False
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [37]:
fig.show()

AttributeError: 'NoneType' object has no attribute 'show'